Le but de se code est de scrapper les prix littéraires en France entre 2019 et 2023

In [89]:
# on importe la bibliothèque requests
import requests
from bs4 import BeautifulSoup # on importe BeautifulSoup
import pandas as pd

On crée un dataframe vide avec comme colonnes les infos qu'on va importer

In [90]:
colonnes = ['Année', 'Prix', 'Auteur', 'Titre']

# Créer un DataFrame vide avec ces colonnes
df = pd.DataFrame(columns=colonnes)

print(df)

Empty DataFrame
Columns: [Année, Prix, Auteur, Titre]
Index: []


On scrape

In [91]:
def scrape_wikipedia_page_france(year):
    global df
    url = f"https://fr.wikipedia.org/wiki/Prix_litt%C3%A9raires_{year}"
    response = requests.get(url)
    response.encoding = "utf-8"  # On garantit le bon encodage

    if response.status_code == 200:  # On vérifie si la requête est un succès
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        # On recherche la section France
        france_section = soup.find('h2', string="France")
        if france_section:
            # La liste <ul> qui suit contient les prix pour la France
            france_list = france_section.find_next('ul')

            if france_list:
                # On recherche tous les <li> dans la liste
                prizes = france_list.find_all('li')

                # On extrait les données
                for prize in prizes:
                    # Nom du prix
                    prize_name = prize.find('a')
                    prize_name = prize_name.text.strip() if prize_name else None

                    # Auteur
                    author = prize.find_all('a')
                    author = author[1].text.strip() if len(author) > 1 else None

                    # Titre
                    title = prize.find('i')
                    title = title.text.strip() if title else None

                    # On ajoute les résultats si les données sont complètes
                    if prize_name and author and title:
                        new_row = {'Année': year, 'Prix': prize_name, 'Auteur': author, 'Titre': title}
                        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        else:
            print(f"Section 'France' non trouvée pour l'année {year}")
    else:
        print(f"Erreur lors du scraping de l'année {year}: Status code {response.status_code}")

    return df

In [92]:
for year in range(2019, 2024):
    scrape_wikipedia_page_france(year)

In [93]:
df.head(5)

,Année,Prix,Auteur,Titre
0,2019,Prix Femina,Par les routes,Par les routes
1,2019,Prix Femina étranger,Ordesa,Ordesa
2,2019,Prix Femina essai,Emmanuelle Lambert,Giono furioso
3,2019,Prix Femina des lycéens,La Chaleur,La Chaleur
4,2019,Prix Goncourt,Tous les hommes n'habitent pas le monde de la ...,Tous les hommes n'habitent pas le monde de la ...


Création d'un fichier CSV afin de vérifier s'il n'y a pas d'incohérences

In [94]:
df.to_csv("prix_litteraires.csv", index=False)

PARTIE: Corriger les incohérences de la table

Il y a un certain nombre d'incohérences dans la table initiales, qui font que notamment les noms d'auteurs sont souvent absents. Parfois, il y est réécrit le titre dans la case dédiée à l'auteur (cas 1). D'autre fois, l'auteur n'est pas indiqué et à la place il y a seulement un chiffre entre crochets (cas 2). Il y a aussi certains prix en particulier qui posent problème à l'image du prix François Maurillac, du prix France-Culture Telerama et du prix Maya.

In [95]:
df_double = df[df['Auteur']==df['Titre']]
print(df_double)

    Année                                         Prix  \
0    2019                                  Prix Femina   
1    2019                         Prix Femina étranger   
3    2019                      Prix Femina des lycéens   
4    2019                                Prix Goncourt   
5    2019               Prix Goncourt du premier roman   
..    ...                                          ...   
189  2022  Grand prix du roman de l'Académie française   
218  2022                          Grand prix RTL-Lire   
223  2022                    Grand Prix Roman de l'été   
290  2023                    Grand Prix Roman de l'été   
292  2023                                   Ada Palmer   

                                                Auteur  \
0                                       Par les routes   
1                                               Ordesa   
3                                           La Chaleur   
4    Tous les hommes n'habitent pas le monde de la ...   
5            

In [96]:
df_ch = df[df['Auteur'].str.contains(r'^\[\d+\]$')]
print(df_ch)

    Année                                  Prix Auteur  \
104  2020              Prix littéraire du Monde    [4]   
150  2021                   Prix Première Plume    [3]   
157  2021  Prix Eugène-Dabit du roman populiste    [5]   
180  2022        Prix Goncourt de la biographie   [10]   
209  2022                           Anne Berest   [33]   
210  2022      Prix littéraire ENS Paris-Saclay   [34]   
214  2022             Prix du Quai des Orfèvres   [38]   
217  2022  Prix Eugène-Dabit du roman populiste   [41]   
222  2022      Grand prix des lectrices de Elle   [45]   
225  2022       Grand Prix de Poésie de la SGDL   [46]   
227  2022                           Prix Wepler   [47]   
247  2023        Prix Goncourt de la biographie   [45]   
297  2023   Prix littéraire de la Ville de Caen   [70]   

                              Titre  
104                           Monde  
150  Avant que le monde ne se ferme  
157              Des gens comme eux  
180           Léopold Sédar Senghor

Cas 3: A la place du nom de l'auteur, il y a toujours indiqué "Académie française"

In [97]:
df_mauriac = df[df['Prix']=='Prix François-Mauriac']
print(df_mauriac)

    Année                   Prix              Auteur  \
53   2019  Prix François-Mauriac  Académie française   
90   2020  Prix François-Mauriac  Académie française   
199  2022  Prix François-Mauriac  Académie française   
267  2023  Prix François-Mauriac  Académie française   

                                  Titre  
53   Là-bas, août est un mois d'automne  
90               Un automne de Flaubert  
199                                Zita  
267   À l’abri des hommes et des choses  


Cas 4: pour le prix Télérama, le nom du prix remplace le titre et le titre remplace l'auteur. Enfin ça dépend desquels donc le remplacement manuel est sûrement mieux pour ces derniers

In [98]:
df_telerama = df[df['Prix']=='Prix France Culture-Télérama']
print(df_telerama)

    Année                          Prix           Auteur     Titre
28   2019  Prix France Culture-Télérama        La Maison  Télérama
97   2020  Prix France Culture-Télérama         Chavirer  Télérama
208  2022  Prix France Culture-Télérama   Kaouther Adimi  Télérama
276  2023  Prix France Culture-Télérama  Salma El Moumni  Télérama


Remplacement à la main

In [99]:
df.loc[(df["Prix"] == "Prix France Culture-Télérama") & (df["Année"] == 2019), ["Auteur", "Titre"]] = ["Emma Becker", "La Maison"]
df.loc[(df["Prix"] == "Prix France Culture-Télérama") & (df["Année"] == 2020), ["Auteur", "Titre"]] = ["Lola Lafon", "Chavirer"]
df.loc[(df["Prix"] == "Prix France Culture-Télérama") & (df["Année"] == 2022), "Titre"] = "Au vent mauvais"
df.loc[(df["Prix"] == "Prix France Culture-Télérama") & (df["Année"] == 2023), "Titre"] = "Adieu Tanger"
nouvelle_ligne = pd.DataFrame({'Année': [2021], 'Prix': ["Prix France Culture-Télérama"], 'Auteur':['Mathieu Palain'], 'Titre':["Ne t'arrête pas de courir "]})
df = pd.concat([df, nouvelle_ligne], ignore_index=True)

Cas 5: Prix Maya

In [100]:
df_maya = df[df['Prix']=='Prix Maya']
print(df_maya)

    Année       Prix      Auteur                       Titre
61   2019  Prix Maya  animaliste  Les Paupières des poissons
124  2020  Prix Maya  animaliste            Insolente Veggie


Il y a aussi un problème avec les prix "Femme actuelle". Il n'y a ni auteur ni titre indiqué, donc il faudra aussi faire à la main.

In [101]:
df_femme_a = df[df['Titre']=='Femme Actuelle']
print(df_femme_a)

    Année                       Prix          Auteur           Titre
46   2019  Grand Prix Roman de l'été  Femme Actuelle  Femme Actuelle
109  2020  Grand Prix Roman de l'été  Femme Actuelle  Femme Actuelle
162  2021  Grand Prix Roman de l'été  Femme Actuelle  Femme Actuelle
223  2022  Grand Prix Roman de l'été  Femme Actuelle  Femme Actuelle
290  2023  Grand Prix Roman de l'été  Femme Actuelle  Femme Actuelle


In [102]:
df.loc[df["Titre"] == "Femme Actuelle", "Prix"] = "Grand prix Roman de l'été de Femme Actuelle"

In [115]:
df.loc[(df['Titre'] == 'Femme Actuelle') & (df['Année'] == 2019), ['Titre', 'Auteur']] = ['Au bout de la nuit', 'Bruno Bouzounie']
df.loc[(df['Titre'] == 'Femme Actuelle') & (df['Année'] == 2020), ['Titre', 'Auteur']] = ['La Conjonction dorée', 'Benoît Sagaro']
df.loc[(df['Titre'] == 'Femme Actuelle') & (df['Année'] == 2021), ['Titre', 'Auteur']] = ['Pour exister encore', 'Martine Duchesne']
df.loc[(df['Titre'] == 'Femme Actuelle') & (df['Année'] == 2022), ['Titre', 'Auteur']] = ["Le journal de Betty Swan", 'Cindy Valt']
df.loc[(df['Titre'] == 'Femme Actuelle') & (df['Année'] == 2023), ['Titre', 'Auteur']] = ['Hauteclaire', 'Dominique Delplace']

Parfois des crochets s'inscrustent dans les titres ou des parenthèses après les noms d'auteur. On veut enlever tout ça !

In [103]:
df['Auteur'] = df['Auteur'].str.replace(r'\(.*\)', '', regex=True) #on enlève les parenthèses après les noms d'auteur
df['Auteur'] = df['Auteur'].str.replace(r'\[.*\]', '', regex=True) # on enlève aussi les éventuels crochets
df['Titre'] = df['Titre'].str.replace(r'\[.*\]', '', regex=True)

RECHERCHE SUR GALLIMARD PCQ TROP DE BLOCAGES SUR LES MOTEURS DE RECHERCHE

In [105]:
import time
import re
import requests
from bs4 import BeautifulSoup as soup

def search_gallimard(book_title, year):
    #reformulation du titre en différente partie
    parts = re.findall(r'\w+|[^\w\s]', book_title)
    book_title_gen = book_title.lower().strip()

    # URL de recherche sur Livraddict
    search_title = '+'.join(parts)
    lower_search_title = '%20'.join(parts).lower()
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:118.0) Gecko/20100101 Firefox/118.0'}
    search_url = f'https://www.gallimard.fr/catalogue?search_api_fulltext={search_title}&field_date_de_parution_1%5Bdate%5D={year}-01-01&field_date_de_parution_2%5Bdate%5D={year}-12-31&field_prix%5Bmin%5D=&field_prix%5Bmax%5D=&title_3=&items_per_page=20&sort_bef_combine=field_identifiant_catalogue_ASC'
    time.sleep(5)

    # Faire une requête pour récupérer la page des résultats de recherche
    response = requests.get(search_url, headers=headers).content
    response_text = response.decode('utf-8')
    # Trouver le premier élément correspondant au lien d'un résultat
    page = soup(response_text, "html.parser")
    results = page.find_all("div", class_ = "card--ouvrage")
    author = ''
    for i in range(len(results)):
        result = results[i]
        div_title = result.find("div", class_ = "title")
        title = div_title.text
        title_gen = title.lower().strip()
        if book_title_gen == title_gen:
            div_author = result.find('div', class_='paragraph paragraph--type--auteur paragraph--view-mode--links')
            a_author = div_author.find("a")
            author = a_author.text
    if author == '':
        author = "Pas trouvé sur Gallimard"
    return author

Fonction pour récupérer l'information sur la maison du livre pour les livres pas sur Gallimard depuis la page wikipedia je pense que je vais enlever tout ça

In [106]:
import time
import re
import requests
from bs4 import BeautifulSoup as soup

def search_maison_du_livre(book_title, year):
    #reformulation du titre en différente partie
    parts = re.findall(r'\w+|[^\w\s]', book_title)
    book_title_gen = book_title.lower().strip()

    # URL de recherche sur Livraddict
    search_title = '+'.join(parts)
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:118.0) Gecko/20100101 Firefox/118.0'}
    search_url = f'https://www.maisondulivre.com/listeliv.php?base=paper&mots_recherche={search_title}+{year}'
    time.sleep(5)

    # Faire une requête pour récupérer la page des résultats de recherche
    response = requests.get(search_url, headers=headers).content
    response_text = response.decode('utf-8')

    # Trouver le premier élément correspondant au lien d'un résultat
    page = soup(response_text, "html.parser")
    liste_livres = page.find('ul', {'id': 'liste_livres'})
    livres = liste_livres.find_all("li")
    author = ''
    for i in range(len(livres)):
        livre = livres[i]
        h2_titre = livre.find("h2", class_="livre_titre")
        if h2_titre:
            a_titre = h2_titre.find("a")
            titre = a_titre.get_text(strip=True)
            title_adapted = re.sub(r'\s?\(.*?\)', '', titre)
            title_gen = title_adapted.lower().strip()
            if title_gen == book_title_gen:
                h2_auteur = livre.find("h2", class_ = "livre_auteur")
                if h2_auteur == None:
                    auteur = "Auteur non renseigné"
                else:
                    a_auteur = h2_auteur.find("a")
                    author = a_auteur.get_text(strip=True)
    if author == '':
        author = "Livre pas trouvé"
    return author

In [107]:
def search_author(row):
    if row["Auteur"] == row["Titre"] or re.match(r'^\[\d+\]$', row['Auteur']) or row['Prix']=='Prix François-Mauriac' or row['Prix']=='Prix Maya':
        author = search_gallimard(row["Titre"], row["Année"])
        if author == None:
            author = "Problème dans la fonction?"
        return author
    else:
        return row["Auteur"]

Fonction globale allant chercher l'auteur de l'ouvrage quand ce dernier n'est pas indiqué. Attention, l'opération prend à peu près 8 minutes

In [108]:
df['Auteur'] = df.apply(search_author, axis=1)

Pour compléter ce qu'on a déjà fait sur Gallimard, on va utiliser un scraping sur le site de la maison du livre

In [109]:
def complete_author(row):
    if row["Auteur"] == "Pas trouvé sur Gallimard":
        author = search_maison_du_livre(row["Titre"], row["Année"])
        if author == None:
            author = "Problème dans la fonction?"
        return author
    else:
        return row["Auteur"]

L'opération prend à peu près 7 minutes 

In [110]:
df['Auteur'] = df.apply(complete_author, axis=1)

In [116]:
df.head(10)

,Année,Prix,Auteur,Titre
0,2019,Prix Femina,Sylvain Prudhomme,Par les routes
1,2019,Prix Femina étranger,Manuel Vilas,Ordesa
2,2019,Prix Femina essai,Emmanuelle Lambert,Giono furioso
3,2019,Prix Femina des lycéens,Victor Jestin,La Chaleur
4,2019,Prix Goncourt,Jean-Paul Dubois,Tous les hommes n'habitent pas le monde de la ...
5,2019,Prix Goncourt du premier roman,Marie Gauthier,Court vêtue
6,2019,Prix Goncourt des lycéens,Karine Tuil,Les Choses humaines
7,2019,Prix Goncourt de la nouvelle,Caroline Lamarche,Nous sommes à la lisière
8,2019,Prix Goncourt de la biographie,Emily Dickinson,"Manifeste incertain, volume 7 : Emily Dickinso..."
9,2019,Choix Goncourt de la Pologne,Louis-philippe Dalembert,Mur Méditerranée


On vérifie qu'il n'y a bien plus aucune anomalie

In [121]:
erreur_recherche = ["Livre pas trouvé"]
df_verif =  df[df["Auteur"].isin(erreur_recherche)]
print(df_verif)

Empty DataFrame
Columns: [Année, Prix, Auteur, Titre]
Index: []


Je remplace à la main pour les dernières valeurs

In [120]:
df.loc[df['Titre'] == "J'ai oublié", 'Auteur'] = 'Bulle Ogier'
df.loc[df['Titre'] == "Une vieille histoire. Nouvelle version", 'Auteur'] = 'Jonathan Littell'
df.loc[df['Titre'] == "Les Grands Cerfs", 'Auteur'] = 'Claudie Hunzinger'
df.loc[df['Titre'] == "Anatèm", 'Auteur'] = 'Neal Stephenson'
df.loc[df['Titre'] == "Le Cœur synthétique", 'Auteur'] = 'Chloé Delaume'
df.loc[df['Titre'] == "Terra Ignota", 'Auteur'] = 'Ada Palmer'
df.loc[df['Titre'] == "Histoire du fils", 'Auteur'] = 'Marie-Hélène Lafon'
df.loc[df['Titre'] == "Adios Cow Boy", 'Auteur'] = 'Olja Savičević Ivančević'
df.loc[df['Titre'] == "La Grâce", 'Auteur'] = 'Thibault de Montaigu'
df.loc[df['Titre'] == "Le Voyant d'Étampes", 'Auteur'] = 'Abel Quentin'
df.loc[df['Titre'] == "Héritage", 'Auteur'] = 'Miguel Bonnefoy'
df.loc[(df['Titre'] == 'Les Paupières des poissons'), 'Auteur'] = 'Fanny Vaucher, Sébastien Moro'
df.loc[df['Titre'] == 'Insolente Veggie', 'Auteur'] = 'Rosa B.'
df.loc[df['Titre'] == 'Zita', 'Auteur'] = 'Olivier Hercend'
df.loc[df['Titre'] == "À l’abri des hommes et des choses", 'Auteur'] = 'Stéphanie Boulay'

Quelques autres corrections

J'ai remarqué qu'il y a parfois des prix qui manquent (c'est deux fois le nom de l'ouvrage). Regarder c'est le cas de combien de livres pour corriger manuellement !

In [131]:
df_verif =  df[(df['Auteur'] == '')]
print(df_verif)

Empty DataFrame
Columns: [Année, Prix, Auteur, Titre]
Index: []


In [129]:

df.loc[(df['Prix'] == 'Prix littéraire du Monde') & (df['Année'] == 2020), ['Titre', 'Auteur']] = ['Elle a menti pour les ailes', 'Francesca Serra']
df.loc[(df['Prix'] == 'Prix Eugène-Dabit du roman populiste') & (df['Année'] == 2021), 'Auteur'] =  'Samira Sedira'
df.loc[(df['Titre'] == 'Mon territoire') & (df['Année'] == 2020), ['Prix', 'Auteur']] = ['Grand prix des lectrices (Elle)', 'Tess Sharpe']
df.loc[df['Titre'] == 'Avant que le monde ne se ferme','Auteur'] = 'Alain Mascaro'
df.loc[df['Titre'] == 'Léopold Sédar Senghor','Auteur'] = ' Jean-Pierre Langellier'
df.loc[df['Titre'] == 'La Carte postale','Auteur'] = 'Anne Berest'
df.loc[df['Titre'] == 'Le Passeur','Auteur'] = 'Lois Lowry'
df.loc[df['Titre'] == 'La Muse rouge','Auteur'] = 'Véronique de Haas'
df.loc[df['Titre'] == 'Les Garçons de la cité-jardin','Auteur'] = 'Dan Nisand'
df.loc[df['Titre'] == 'Réacteur 3 ','Auteur'] = 'Ludovic Bernhardt'
df.loc[df['Titre'] == 'Les Enfants endormis','Auteur'] = 'Anthony Passeron'
df.loc[df['Titre'] == 'Georges Perec','Auteur'] = 'Claude Burgelin'
df.loc[df['Titre'] == "Mourir avant que d'apparaître",'Auteur'] = 'Rémi David'


Pour ces lignes, on a pas besoin de changer parce qu'on exploite la base en utilisant une indicatrice de prix ( donc ce n'est pas important pour nous de savoir quel prix exactement a été obtenu)

On vérifie que tout va bien dans le csv

In [132]:
df.to_csv("prix_litteraires.csv", index=False)

In [133]:
import pandas as pd

# on ajoute une colonne indicatrice "prix" 
df['prix_19_20'] = 1 # renommer en 19_23 pour plus de clarté ?
print("Colonnes avant sauvegarde :", df.columns)

#tentative de sauvegarde en csv
df.to_csv("prix_litteraires.csv", index=False, mode="w")

print("fin")

Colonnes avant sauvegarde : Index(['Année', 'Prix', 'Auteur', 'Titre', 'prix_19_20'], dtype='object')
fin
